In [2]:
# For Data Processing
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.utils import shuffle


# For ML Models
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import load_img
import cv2

import os
import pickle

model_file = 'cnn_model2.sav'

training_folder = 'Training'
testing_folder = 'Testing'

training_images_paths = []
training_images_target = []


# Our training model path is  Training_
for label in os.listdir(training_folder):             
    for image in os.listdir(training_folder+"/" +label):
        training_images_paths.append(training_folder + '/'+label+ '/'+image)
        if(label=='YES'):
            training_images_target.append(1)
        elif(label=='NO'):
            training_images_target.append(0)


            
testing_images_paths = []
testing_images_target = []

for label in os.listdir(testing_folder):
    for image in os.listdir(testing_folder+ '/' + label):
        testing_images_paths.append(testing_folder +'/'+label + '/'+image)
        if(label=='YES'):
            testing_images_target.append(1)
        elif(label=='NO'):
            testing_images_target.append(0)

            
testing_images_paths, testing_images_target = shuffle(testing_images_paths, testing_images_target)


training_images_paths, training_images_target = shuffle(training_images_paths, training_images_target)



C:\Users\MOEIZ.XTREMELITE-PC\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:



def extracting_images(paths):

    images = []
    for path in paths:
        image = load_img(path, target_size=(200, 200))
        image = Image.fromarray(np.uint8(image))
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)


print("Done")

Done


In [29]:

model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))




model.add(Conv2D(36, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()
print("Done")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 99, 99, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 97, 97, 36)        5220      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 48, 48, 36)       0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 82944)             0         
                                                                 
 dense_20 (Dense)            (None, 128)             

In [32]:


def batch_creation(paths, labels, batch_size, epochs):
    for i in range(0, epochs):

        for pathindex in range(0, len(paths), batch_size):
            batch_paths = paths[pathindex:pathindex+batch_size] #This would create batches of given size
            batch_images = extracting_images(batch_paths)
            batch_labels = labels[pathindex:pathindex+batch_size]
            batch_labels=np.array(batch_labels)
            yield batch_images, batch_labels

model.fit(batch_creation(training_images_paths, training_images_target, batch_size=32,epochs=5),steps_per_epoch=178, epochs=5)
pickle.dump(model, open(model_file, 'wb'))

Epoch 1/5
178/178 [==============================] - 132s 741ms/step - loss: 0.0221 - accuracy: 0.9917
Epoch 2/5
178/178 [==============================] - 132s 744ms/step - loss: 0.0264 - accuracy: 0.9884
Epoch 3/5
178/178 [==============================] - 134s 752ms/step - loss: 0.0160 - accuracy: 0.9935
Epoch 4/5
178/178 [==============================] - 134s 755ms/step - loss: 0.0175 - accuracy: 0.9931
Epoch 5/5
178/178 [==============================] - 133s 747ms/step - loss: 0.0104 - accuracy: 0.9947
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......v

In [5]:
loaded_model = pickle.load(open(model_file, 'rb'))

predictions=loaded_model.predict(extracting_images(testing_images_paths))
model_test_predictions=[]

for i in range(0,len(predictions)):
    if predictions[i]>=0.5:
        model_test_predictions.append(1)
    else:
        model_test_predictions.append(0)


print(classification_report(model_test_predictions,testing_images_target))


Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-12-25 21:27:00         3246
metadata.json                                  2022-12-25 21:27:00           64
variables.h5                                   2022-12-25 21:27:00    127507000
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2

In [8]:
path="F:/Classification Project/training_set/hemmorhage_data/098.png"


predictionimages = [] #The model takes a numpy array
image = load_img(path, target_size=(200, 200))
image = Image.fromarray(np.uint8(image))

image = np.array(image)/255.0

predictionimages.append(image)
predictionimages=np.array(predictionimages)


predictions = loaded_model.predict(predictionimages)
if(predictions[0][0]<0.5):
    print("Tumor was not detected")
elif(predictions[0][0]>=0.5):
    print("Tumor was detected")

[[[0.17647059 0.17647059 0.17647059]
  [0.17254902 0.17254902 0.17254902]
  [0.17254902 0.17254902 0.17254902]
  ...
  [0.19215686 0.19215686 0.19215686]
  [0.18823529 0.18823529 0.18823529]
  [0.19215686 0.19215686 0.19215686]]

 [[0.18431373 0.18431373 0.18431373]
  [0.18431373 0.18431373 0.18431373]
  [0.18431373 0.18431373 0.18431373]
  ...
  [0.18823529 0.18823529 0.18823529]
  [0.19215686 0.19215686 0.19215686]
  [0.20392157 0.20392157 0.20392157]]

 [[0.18823529 0.18823529 0.18823529]
  [0.18431373 0.18431373 0.18431373]
  [0.18039216 0.18039216 0.18039216]
  ...
  [0.2        0.2        0.2       ]
  [0.19215686 0.19215686 0.19215686]
  [0.19607843 0.19607843 0.19607843]]

 ...

 [[0.51372549 0.51372549 0.51372549]
  [0.76862745 0.76862745 0.76862745]
  [0.78431373 0.78431373 0.78431373]
  ...
  [0.19607843 0.19607843 0.19607843]
  [0.20392157 0.20392157 0.20392157]
  [0.21568627 0.21568627 0.21568627]]

 [[0.61176471 0.61176471 0.61176471]
  [0.98431373 0.98431373 0.98431373]


178


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-12-25 21:27:00         3246
metadata.json                                  2022-12-25 21:27:00         